In [ ]:
import pandas as pd
import numpy as np

def texts_to_corpus(temp_text):
    uni_words = np.unique(np.array(temp_text))
    corpus = []
    for uni_word in uni_words:
        corpus.append((uni_word,temp_text.count(uni_word)))
    return corpus

def test2str(t_texts):
    str_text = []
    for t_text in t_texts:
        str_text.append(str(t_text))
    return str_text

third_stage = pd.read_csv(r'\clean_1Hz(new).csv')

ids = np.unique(third_stage['id'].to_numpy())

texts = []
for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    temp_words = single_data['words'].values
    #print(temp_words)
    temp_text = []
    for temp_word in temp_words:
        temp_text.append(str(temp_word))
    #print(temp_text)
    texts.append(temp_text)


In [ ]:
from gensim import corpora, models, similarities
#import jieba.posseg as jp, jieba

dictionary = corpora.Dictionary.load(r'\dic_new.dict')
corpus = [dictionary.doc2bow(words) for words in texts]
# print(len(corpus))
# print(dictionary,dictionary[0])

In [ ]:
tfidf_model = models.TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]
print(len(corpus_tfidf))

In [ ]:
from gensim.test.utils import common_corpus
from gensim.models.ldamodel import LdaModel
from gensim.test.utils import datapath
from gensim import models, corpora

ldamodel = models.ldamodel.LdaModel.load(r"\auto_model_new_test.model")

#test the effectiveness of ldamodel

test_cor = dictionary.doc2bow(['683','528','501','714','497','714','683'])
#test_cor = [words_to_cor(683),words_to_cor(528),words_to_cor(501),words_to_cor(714),words_to_cor(497),words_to_cor(714),words_to_cor(683)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

test_cor = dictionary.doc2bow(['657','831','840','863','687','869','807'])
#test_cor = [words_to_cor(657),words_to_cor(831),words_to_cor(840),words_to_cor(863),words_to_cor(687),words_to_cor(869),words_to_cor(807)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

test_cor = dictionary.doc2bow(['499','683','714','684','558','528','528'])
#test_cor = [words_to_cor(499),words_to_cor(683),words_to_cor(714),words_to_cor(684),words_to_cor(558),words_to_cor(528),words_to_cor(528)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

test_cor = dictionary.doc2bow(['373','404','404','373','404','373','372'])
#test_cor = [words_to_cor(373),words_to_cor(404),words_to_cor(404),words_to_cor(373),words_to_cor(404),words_to_cor(373),words_to_cor(372)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

#test_cor = [(528, 6), (558, 1)]
test_cor = dictionary.doc2bow(['528','528','528','528','528','528','558'])
#test_cor = [words_to_cor(373),words_to_cor(404),words_to_cor(404),words_to_cor(373),words_to_cor(404),words_to_cor(373),words_to_cor(372)]
print(test_cor,ldamodel[tfidf_model[test_cor]])

In [ ]:
def search_known(prediction):
    known_topics = []
    knowm_fea = []
    for i in range(len(prediction)):
        known_topics.append(prediction[i][0])
        knowm_fea.append(prediction[i][1])
    return known_topics,knowm_fea

def construct_feature(prediction,num_topic):
    known_topics,knowm_fea = search_known(prediction)
    #print(known_topics)
    temp_feature = []
    if len(known_topics)==num_topic:
        return knowm_fea
    for i in range(num_topic):
        completion = (1-sum(knowm_fea))/(num_topic-len(known_topics))
        if i in known_topics:
            indexs = known_topics.index(i)
            temp_feature.append(knowm_fea[indexs])
        else:
            temp_feature.append(completion)
    return temp_feature

def test2str(t_texts):
    str_text = []
    for t_text in t_texts:
        str_text.append(str(t_text))
    return str_text

In [ ]:
# multi-step data preprocess

ids = np.unique(third_stage['id'].to_numpy())
total_times = 0
num_topic = 5
train_data = []
single_length = []

for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    
    for i in range(len(single_data)-6):
        #for j in range(i,i+7):
        temp_text = [single_data.loc[i]['words'],single_data.loc[i+1]['words'],single_data.loc[i+2]['words'],
                    single_data.loc[i+3]['words'],single_data.loc[i+4]['words'],single_data.loc[i+5]['words'],
                    single_data.loc[i+6]['words']]
        
        tuple_corpus = dictionary.doc2bow(test2str(temp_text))
        temp_feature = construct_feature(ldamodel[tfidf_model[tuple_corpus]],num_topic)
        ID = single_data.loc[i+6]['id']
        t = single_data.loc[i+6]['t']
        corpus = dictionary.doc2bow([str(single_data.loc[i+6]['words'])])
        his_text = [single_data.loc[i+1]['words'],single_data.loc[i+2]['words'],
                    single_data.loc[i+3]['words'],single_data.loc[i+4]['words'],single_data.loc[i+5]['words'],
                    single_data.loc[i+6]['words']]
        train_data.append([ID,t,len(single_data)-6,corpus,temp_feature,int(single_data.loc[i+6]['words']),his_text])
    total_times += len(single_data)-6
    single_length.append(len(single_data)-6)

In [ ]:
def pad_seq(temp_train):
    np_train = np.array(temp_train)
    if np_train.shape[0]>14:
        print('error')
        return np_train
    elif np_train.shape[0]==14:
        return np_train
    else:
        loss_len = 14-np_train.shape[0]
        loss_train = np.zeros([loss_len,5])
        return np.concatenate([loss_train,np_train],axis=0)

ids = np.unique(third_stage['id'].to_numpy())
flag = 0
minimum = 4
maximum = 14
all_train = []
train_speed = []
train_topic = []
history_speed = []
np_all_train = np.zeros([0,14,5])

for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    length = single_length[int(ID)-1]
    temp_data = train_data[flag:flag+length]
    flag += length
    if length<4+15:
        continue
    for i in range(length-minimum+1-15):
        end_num = min(length-15,i+maximum)
        #print(i,end_num)
        for j in range(i+minimum-1,end_num):
            next_speed = [temp_data[j+1][5],temp_data[j+2][5],temp_data[j+3][5],temp_data[j+4][5],temp_data[j+5][5],temp_data[j+6][5],temp_data[j+7][5],temp_data[j+8][5],temp_data[j+9][5],temp_data[j+10][5],temp_data[j+11][5],temp_data[j+12][5],temp_data[j+13][5],temp_data[j+14][5],temp_data[j+15][5]]
            next_topic = [temp_data[j+1][4],temp_data[j+2][4],temp_data[j+3][4],temp_data[j+4][4],temp_data[j+5][4],temp_data[j+6][4],temp_data[j+7][4],temp_data[j+8][4],temp_data[j+9][4],temp_data[j+10][4],temp_data[j+11][4],temp_data[j+12][4],temp_data[j+13][4],temp_data[j+14][4],temp_data[j+15][4]]
            train_speed.append(next_speed)
            train_topic.append(next_topic)
            history_speed.append(temp_data[j][6])
            temp_train = []
            for k in range(i,j+1):
                temp_train.append(temp_data[k][4])
            np_temp_train = pad_seq(temp_train).reshape([-1,pad_seq(temp_train).shape[0],pad_seq(temp_train).shape[1]])
            all_train.append(temp_train)
            np_all_train = np.concatenate([np_all_train,np_temp_train],axis=0)
#print(single_data)

In [ ]:
np_train_speed = np.array(train_speed)
np_train_topic = np.array(train_topic)
#np_history_speed = np.array(history_speed)
print(np_all_train.shape)
print(np_train_speed.shape,np_train_topic.shape)

np_history_speed = np.zeros([len(history_speed),6])
for i in range(len(history_speed)):
    for j in range(len(history_speed[i])):
        np_history_speed[i,j] = history_speed[i][j]
        
np.save(r'\np_train_speed_multistep_new.npy',np_train_speed)
np.save(r'\np_train_topic_multistep_new.npy',np_train_topic)
np.save(r'\np_all_train_multistep_new.npy',np_all_train)
np.save(r'\np_history_speed_multistep_new.npy',np_history_speed)

In [ ]:
# multi-step topic prediction

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import sampler, Dataset, DataLoader

class MultiTopicData(Dataset):
    def  __init__(self,inputs,labels):
        self.inputs=torch.DoubleTensor(inputs.astype(float)).to(torch.float32)
        self.labels=torch.DoubleTensor(labels.astype(float)).to(torch.float32)
        
        self.len = inputs.shape[0]
    
    def __getitem__(self,index):
        inp = self.inputs[index,:,:]
        label = self.labels[index,:,:]
        
        return inp,label
    
    def __len__(self):
        return self.len

x = np_all_train
y = np_train_topic

shuffled_indices = np.random.permutation(len(np_all_train))
#train_indices = shuffled_indices[:int(len(np_all_train)*0.8)]
#val_indices = shuffled_indices[int(len(np_all_train)*0.8):int(len(np_all_train)*0.9)]
test_indices = shuffled_indices[int(len(np_all_train)*0.90):]
# test_indices_1 = shuffled_indices[int(len(np_all_train)*0.90):int(len(np_all_train)*0.95)]
# test_indices_2 = shuffled_indices[int(len(np_all_train)*0.95):]

#x_train = x[train_indices,:,:]
#y_train = y[train_indices,:]
#x_val = x[val_indices,:,:]
#y_val = y[val_indices,:]
x_test = x[test_indices,:,:]
y_test = y[test_indices,:]
# x_test_1 = x[test_indices_1,:,:]
# y_test_1 = y[test_indices_1,:]
# x_test_2 = x[test_indices_2,:,:]
# y_test_2 = y[test_indices_2,:]

#print(x_train.shape,y_train.shape)
#print(x_val.shape,y_val.shape)
print(x_test.shape,y_test.shape)
# print(x_test_1.shape,y_test_1.shape)
# print(x_test_2.shape,y_test_2.shape)

#trainset = TopicData(x_train, y_train)
#train_data_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)
#valset = TopicData(x_val, y_val)
#val_data_loader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
topic_testset = MultiTopicData(x_test,y_test)
topic_test_data_loader = DataLoader(topic_testset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)

# topic_testset_1 = MultiTopicData(x_test_1,y_test_1)
# topic_test_data_loader_1 = DataLoader(topic_testset_1, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
# topic_testset_2 = MultiTopicData(x_test_2,y_test_2)
# topic_test_data_loader_2 = DataLoader(topic_testset_2, batch_size=64, shuffle=False, num_workers=0, drop_last=True)


In [ ]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import numpy as np
from spikingjelly.clock_driven import neuron, encoding, functional
#from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Define and initialize the network
tau=2.0
num_topic = 5
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(14*num_topic, 256, bias=False),
    nn.Linear(256, 512, bias=False),
    nn.LayerNorm(512),
    nn.Linear(512, 256, bias=False),
    nn.Linear(256, num_topic, bias=False),
    neuron.LIFNode(tau=tau)
)
model = model.to('cpu')
encoder = encoding.PoissonEncoder()

model.load_state_dict(torch.load(r'\SNN_topic_new.tar'))

In [ ]:
#multi step topic prediction
test_sum = [0,0,0,0,0,0,0,0,0,0]
correct_sum = [0,0,0,0,0,0,0,0,0,0]
test_loss = [0,0,0,0,0,0,0,0,0,0]
T = 50
all_outputs = torch.zeros([0,10,5])

for batch_idx, (inputs, labels) in enumerate(topic_test_data_loader):
    #inputs = inputs.to(device)
    #labels = labels.to(device)
    if batch_idx%10==0:
        print(batch_idx*inputs.shape[0]/len(x_test)*100)
    
    all_inputs = inputs
    time_step = 0
    for time_step in range(10):
        for t in range(T):
            if t == 0:
                out_spikes_counter = model(encoder(all_inputs[:,-14:,:]).float())
            else:
                out_spikes_counter += model(encoder(all_inputs[:,-14:,:]).float())
        
        out_spikes_counter = out_spikes_counter / T
        correct_sum[time_step] += (out_spikes_counter.max(1)[1] == labels[:,time_step,:].max(1)[1]).float().sum().item()
        loss = F.mse_loss(out_spikes_counter, labels[:,time_step,:])
        test_loss[time_step] += loss.item()*inputs.shape[0]
        test_sum[time_step] += inputs.shape[0]
        
        all_inputs = torch.cat([all_inputs,out_spikes_counter.unsqueeze(1)],dim=1)
    all_outputs = torch.cat([all_outputs,all_inputs[:,-15:,:]])

In [ ]:
for t in range(10):
    print('{}s top 1 topic accuracy: {:.2f}%'.format(t+1,correct_sum[t]/test_sum[t]*100))
for t in range(10):
    print('{}s RMSE Loss: {:.2f}'.format(t+1,np.sqrt(test_loss[t]/test_sum[0])))

In [ ]:
#multi-step speed prediction

bow_train_speeds = np.zeros([np_train_speed.shape[0],np_train_speed.shape[1]],dtype=int)
#bow_history_speeds = np.zeros([np_train_speed.shape[0],np_train_speed.shape[1]],dtype=int)

for idx,temp_speed in enumerate(np_train_speed):
    #print(temp_speed)
    temp_bow_train_speed = []
    for each_speed in test2str(temp_speed):
        temp_bow_train_speed.append(int(dictionary.doc2bow([each_speed])[0][0]))
    bow_train_speeds[[idx],:] = np.array(temp_bow_train_speed,dtype=int).reshape(1,-1)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import sampler, Dataset, DataLoader

class MultiSpeedData(Dataset):
    def  __init__(self,inputs,inputs_speed,labels):
        self.inputs=torch.DoubleTensor(inputs.astype(float)).to(torch.float32)
        self.inputs_speed=torch.DoubleTensor(inputs_speed.astype(float)).to(torch.float32)
        self.labels=torch.DoubleTensor(labels.astype(float)).to(torch.float32)
        
        self.len = inputs.shape[0]
        
    def __getitem__(self,index):
        inp = self.inputs[index,:,:]
        input_speed = self.inputs_speed[index,:]
        label = self.labels[index,:]
        
        return inp,input_speed,label
        
    def __len__(self):
        return self.len
    
x = np_all_train
x_speed = np_history_speed
y = bow_train_speeds

shuffled_indices = np.random.permutation(len(np_all_train))
# train_indices = shuffled_indices[:int(len(np_all_train)*0.8)]
# val_indices = shuffled_indices[int(len(np_all_train)*0.8):int(len(np_all_train)*0.9)]
test_indices = shuffled_indices[int(len(np_all_train)*0.9):]
# test_indices_2 = shuffled_indices[int(len(np_all_train)*0.95):]
# test_indices_1 = shuffled_indices[int(len(np_all_train)*0.90):int(len(np_all_train)*0.95)]

# x_train = x[train_indices,:,:]
# y_train = y[train_indices,:]
# x_val = x[val_indices,:,:]
# y_val = y[val_indices,:]
x_test = x[test_indices,:,:]
x_speed_test = x_speed[test_indices,:]
y_test = y[test_indices,:]
# x_test_1 = x[test_indices_1,:,:]
# x_speed_test_1 = x_speed[test_indices_1,:]
# y_test_1 = y[test_indices_1,:]
# x_test_2 = x[test_indices_2,:,:]
# x_speed_test_2 = x_speed[test_indices_2,:]
# y_test_2 = y[test_indices_2,:]

#print(x_train.shape,y_train.shape)
#print(x_val.shape,y_val.shape)
print(x_test.shape,y_test.shape)
# print(x_test_1.shape,y_test_1.shape)
# print(x_test_2.shape,y_test_2.shape)

#trainset = TopicData(x_train, y_train)
#train_data_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)
#valset = TopicData(x_val, y_val)
#val_data_loader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
testset = MultiSpeedData(x_test,x_speed_test,y_test)
test_data_loader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=0, drop_last=True)
# testset_1 = MultiSpeedData(x_test_1,x_speed_test_1,y_test_1)
# test_data_loader_1 = DataLoader(testset_1, batch_size=1, shuffle=False, num_workers=0, drop_last=True)
# testset_2 = MultiSpeedData(x_test_2,x_speed_test_2,y_test_2)
# test_data_loader_2 = DataLoader(testset_2, batch_size=1, shuffle=False, num_workers=0, drop_last=True)


In [ ]:
import math
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def array_to_cor(inputs_speed):
    array_speed = inputs_speed.cpu().numpy()
    cor_speed = []
    for i in range(6):
        cor_speed.append((int(array_speed[0,2*i+0]),int(array_speed[0,2*i+1])))
    return cor_speed

def words_to_cor(words):
    a = math.ceil(words/30)
    b = words % 30
    return (a,b)

def cor_to_words(cor):
    a,b = cor
    words = (a-1)*30.0+b
    return words

def lon_speed(lon):
    if lon != 6:
        return 2+(lon-1)*4
    else:
        return 22

def lat_speed(lat):
    if lat == 1:
        return -0.4479
    elif lat == 2:
        return -0.2240
    elif lat == 3:
        return 0
    elif lat == 4:
        return 0.2240
    elif lat == 5:
        return 0.4479
    
def cor_to_speed(cor):
    a,b = cor
    a_lon = math.ceil(a/5)
    a_lat = a - (a_lon-1)*5
    b_lon = math.ceil(b/5)
    b_lat = b - (b_lon-1)*5
    
    return [a_lon,a_lat,b_lon,b_lat],[lon_speed(a_lon),lon_speed(b_lon)],[lat_speed(a_lat),lat_speed(b_lat)]

def batch_speed_trans(outputs):
    #outputs = outputs.item()
    lons = np.zeros([0,2])
    lats = np.zeros([0,2])
    for output in outputs:
        _,lon_v_pre,lat_v_pre = cor_to_speed(words_to_cor(output))
        lons = np.concatenate([lons,np.array(lon_v_pre).reshape(1,-1)],axis=0)
        lats = np.concatenate([lats,np.array(lat_v_pre).reshape(1,-1)],axis=0)
    return lons,lats

def hot2doc(out_spikes_counter_max,dictionary):
    docs = []
    for hot_code in out_spikes_counter_max:
        docs.append(int(dictionary[hot_code.item()]))
    return docs

In [ ]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import numpy as np
from spikingjelly.clock_driven import neuron, encoding, functional
#from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Define and initialize the network
tau=5.0
num_topic = 5
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(14*num_topic, 256, bias=False),
    nn.Linear(256, 512, bias=False),
    nn.LayerNorm(512),
    nn.Linear(512, 256, bias=False),
    nn.Linear(256, 307, bias=False),
    neuron.LIFNode(tau=tau)
)
model = model.to('cpu')
encoder = encoding.PoissonEncoder()

model.load_state_dict(torch.load(r'\SNN_speed_new.tar'))

In [ ]:
#multi step speed prediction
test_sum = 0
correct_sum = 0
test_loss = 0
T = 20
num_topic = 5
all_outputs_lons = np.zeros([0,30])
all_outputs_lats = np.zeros([0,30])
all_labels_lons = np.zeros([0,30])
all_labels_lats = np.zeros([0,30])
for batch_idx, (inputs,inputs_speed,labels) in enumerate(test_data_loader):
    if batch_idx%10==0:
        print(batch_idx*inputs.shape[0]/len(x_test)*100)
    
    history_speed = list(np.array(inputs_speed.cpu().numpy()[0],dtype=int))
    labels = labels.squeeze(1)
    
    temp_outputs_lons = np.zeros([1,30])
    temp_outputs_lats = np.zeros([1,30])
    temp_labels_lons = np.zeros([1,30])
    temp_labels_lats = np.zeros([1,30])
    
    temp_steps = 0
    step_inputs = inputs
    
    for temp_steps in range(10):
        for t in range(T):
            if t == 0:
                out_spikes_counter = model(encoder(step_inputs).float())
            else:
                out_spikes_counter += model(encoder(step_inputs).float())
        out_spikes_counter = out_spikes_counter / T
    
        output_onestep = out_spikes_counter.max(1)[1]
        outputs_lons,outputs_lats = batch_speed_trans(hot2doc(output_onestep,dictionary))
        labels_lons,labels_lats = batch_speed_trans(hot2doc(labels[0,temp_steps].unsqueeze(0),dictionary))
        temp_outputs_lons[:,temp_steps*2:temp_steps*2+2] = outputs_lons
        temp_outputs_lats[:,temp_steps*2:temp_steps*2+2] = outputs_lats
        temp_labels_lons[:,temp_steps*2:temp_steps*2+2] = labels_lons
        temp_labels_lats[:,temp_steps*2:temp_steps*2+2] = labels_lats
        
        history_speed.append(int(dictionary[output_onestep.item()]))
        tuple_corpus = dictionary.doc2bow(test2str(history_speed[-7:]))
        temp_feature = construct_feature(ldamodel[tfidf_model[tuple_corpus]],num_topic)
        temp_feature = np.array(temp_feature).astype(float)
        step_inp = torch.DoubleTensor(temp_feature).to(torch.float32).reshape([1,1,-1])
        step_inputs = torch.cat([step_inputs,step_inp],dim=1)[:,-14:,:]
    
    all_outputs_lons = np.concatenate([all_outputs_lons,temp_outputs_lons],axis=0)
    all_outputs_lats = np.concatenate([all_outputs_lats,temp_outputs_lats],axis=0)
    all_labels_lons = np.concatenate([all_labels_lons,temp_labels_lons],axis=0)
    all_labels_lats = np.concatenate([all_labels_lats,temp_labels_lats],axis=0)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

for t in range(10):
    print('{}s prediction error : '.format(t+1))
    print(f"Lon_v: MSE：{mean_squared_error(all_outputs_lons[:,2*t:2*t+2], all_labels_lons[:,2*t:2*t+2])}")
    print(f"Lon_v: RMSE：{np.sqrt(mean_squared_error(all_outputs_lons[:,2*t:2*t+2], all_labels_lons[:,2*t:2*t+2]))}")
    print(f"Lon_v: MAE：{mean_absolute_error(all_outputs_lons[:,2*t:2*t+2], all_labels_lons[:,2*t:2*t+2])}")
    print(f"Lat_v: MSE：{mean_squared_error(all_outputs_lats[:,2*t:2*t+2], all_labels_lats[:,2*t:2*t+2])}")
    print(f"Lat_v: RMSE：{np.sqrt(mean_squared_error(all_outputs_lats[:,2*t:2*t+2], all_labels_lats[:,2*t:2*t+2]))}")
    print(f"Lat_v: MAE：{mean_absolute_error(all_outputs_lats[:,2*t:2*t+2], all_labels_lats[:,2*t:2*t+2])}")
    print()